# Carregando o DataFrame - Causas de mortes em países ao redor do mundo dos anos de 1990 à 2019.

In [2]:
import pandas as pd
import numpy as np


In [ ]:
causas_mortes = pd.read_csv('cause_of_deaths.csv')
print(causas_mortes)

# Removendo valores duplicados da coluna de "Country/Territory". Serão contabilizados somente os registros dos anos de 1990 para cada país.

In [ ]:
causas_mortes = causas_mortes.drop_duplicates(subset= 'Country/Territory')
print(causas_mortes)

In [ ]:
causas_mortes.info()

# Criando a matriz numérica para o calculo da correlação.

In [ ]:
variaveis_numericas = causas_mortes.drop(columns= ['Country/Territory', 'Code', "Year"])
variaveis_numericas.corr()

# Verificando as condições dos dados para aplicar o PCA

In [ ]:
!pip install factor_analyzer

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo

In [ ]:
kmo_variaveis, kmo = calculate_kmo(variaveis_numericas)

In [ ]:
kmo_variaveis

In [ ]:
kmo

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

In [ ]:
qui_quadrado, pvalor = calculate_bartlett_sphericity(variaveis_numericas)

In [ ]:
print(qui_quadrado)
print(pvalor)

In [ ]:
pvalor < 0.05

# Padronizando as variáveis numéricas

In [9]:
from sklearn.preprocessing import StandardScaler

In [ ]:
colunas_numericas = variaveis_numericas.columns
colunas_numericas

In [ ]:
padronizar = StandardScaler()
variaveis_numericas = padronizar.fit_transform(variaveis_numericas)
variaveis_numericas

In [ ]:
dados_padronizados = pd.DataFrame(variaveis_numericas, columns= colunas_numericas)
dados_padronizados

# Aplicando o PCA para todos os possíveis fatores

In [36]:
from sklearn.decomposition import PCA

In [38]:
n_fatores = dados_padronizados.shape[1]
n_fatores

31

In [40]:
pca = PCA(n_components= n_fatores)
pca.fit(dados_padronizados)

PCA(n_components=31)

## Checando os Autovalores, Autovetores e a Variância explicada pelos fatores

In [ ]:
print("Auto-valores:")
print(pca.explained_variance_,'\n')

print("Auto-vetores:")
print(pca.components_,'\n')

print("Variância explicada:")
print(pca.explained_variance_ratio_,'\n')

In [ ]:
import plotly.express as px
import numpy as np

In [ ]:
fatores = [f'F{i+1}' for i in range(n_fatores)]
fatores

In [ ]:
fig = px.bar (x= fatores, y= pca.explained_variance_ratio_, text= np.around(pca.explained_variance_ratio_, decimals= 2), title= 'Scree Plot' )
fig.update_layout(yaxis = { 'title': 'Porcentagem de variância explicada', 'tickfont' : { 'size' : 15}},
                  xaxis= {'title' : 'Fatores', 'tickfont' : {'size' : 15}}, 
                  title = {'font' : {'size' : 25}})
fig.show()

In [ ]:
variancia_acumulada = [sum(pca.explained_variance_ratio_[0:i+1]) for i in range(n_fatores)]
variancia_acumulada


# Verificando os dois maiores Fatores

In [ ]:
pca = PCA(n_components= 2)
pca.fit(dados_padronizados)

PCA(n_components=2)

In [ ]:
fatores2 = [f'F{i+1}' for i in range(pca.n_components)]
fatores2

['F1', 'F2']

In [ ]:
fig = px.bar (x= fatores2, y= pca.explained_variance_ratio_, text= np.around(pca.explained_variance_ratio_, decimals= 2), title= 'Scree Plot' )
fig.update_layout(yaxis = { 'title': 'Porcentagem de variância explicada', 'tickfont' : { 'size' : 15}},
                  xaxis= {'title' : 'Fatores', 'tickfont' : {'size' : 15}}, 
                  title = {'font' : {'size' : 25}})
fig.show()